In [69]:
from selenium import webdriver;
from selenium.webdriver.common.keys import Keys;
import time
from datetime import datetime, timedelta
import re

In [48]:
driver = webdriver.Chrome()

In [49]:
url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do"
driver.get(url)
time.sleep(15)

In [50]:
target_date = "2024-10-01"

In [51]:
## 함수 모음
def increase_date(start_date):
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    increased_date = start_datetime + timedelta(days=1)
    return increased_date

def set_target_date(target_date):
    formatted_date = target_date[2:]
    search_start_date = driver.find_element_by_css_selector('div.tf_comm > input#sSearchFrom')
    for i in range(0, 8):
        search_start_date.send_keys(Keys.BACK_SPACE)
    search_start_date.send_keys(formatted_date)
    
    search_start_date = driver.find_element_by_css_selector('div.tf_comm > input#sSearchTo')
    for i in range(0, 8):
        search_start_date.send_keys(Keys.BACK_SPACE)
    search_start_date.send_keys(formatted_date)
    
def click_more_button():
    while True:
        try:
            more_button = driver.find_element_by_css_selector('div.more  > a.btn_more')
            more_button.click()
        except:
            break
        
def get_movie_info():
    movie_table = driver.find_element_by_css_selector('tbody#tbody_0')

    rows = movie_table.find_elements_by_tag_name('tr')
    first_row = rows[0]
    row_info = first_row.find_elements_by_tag_name('td')

    rank = row_info[0].text
    # title = row_info[1].text
    sales = row_info[3].text
    audience = row_info[7].text

    # print("rank = " + rank)
    # print("sales = " + sales)
    # print("audience = " + audience)


In [52]:
set_target_date(target_date)
search_button = driver.find_element_by_css_selector('div.wrap_btn > button.btn_blue')
search_button.click()
click_more_button()
get_movie_info()

rank = 1
sales = 1,732,773,470
audience = 166,508


In [55]:
# 영화 상세 정보를 얻기 위헤 제목 클릭
second_td = first_row.find_elements_by_tag_name('td')[1]
movie_info_link = second_td.find_element_by_css_selector('span.ellip.per90 > a')
movie_info_link.click()

In [56]:
# 영화를 클릭하면 나오는 상세 정보 템플릿에서 영화 메타데이터 가져오기
movie_info_template = driver.find_element_by_css_selector('div[tabindex="-1"]')

title = movie_info_template.find_element_by_css_selector('div.hd_layer > div').text.replace("영화상영관상영중", "").strip()
movie_info = movie_info_template.find_element_by_css_selector('dl.ovf')
metadata = movie_info.find_elements_by_css_selector('dt, dd')
code = metadata[1].text

movie_details = metadata[7].text
release_date = metadata[11].text
production_year = metadata[13].text

print("code = " + code)
print("movie_details = " + movie_details)
print("release_date = " + release_date)
print("production_year = " + production_year)

code = 20249434
movie_details = 장편 | 일반영화 | 범죄, 드라마, 뮤지컬 | 137분 49초 | 15세이상관람가 | 미국
release_date = 2024-10-01
production_year = 2024년


In [57]:
# 코드 정보를 바탕으로 감독, 스태프가 있는 태그에 접근해 정보 얻기
synopsys = movie_info_template.find_element_by_css_selector('p.desc_info').text
print("synopsys = " + synopsys)

synopsys = “더 이상 혼자가 아니야”
세상이란 무대에서 폭주하기 시작한 ‘조커’와 ‘할리 퀸’을 확인하라!

2년 전, 세상을 뒤흔들며 고담시 아이콘으로 자리한 ‘아서 플렉’은
아캄 수용소에 갇혀 최종 재판을 앞둔 무기력한 삶을 살아간다.

그러던 어느 날, 수용소에서 운명적으로 만난 ‘리 퀸젤’은
‘아서’의 삶을 다시 뒤바꾸며 그의 마음 속에 잠들어 있던 ‘조커’를 깨우고
‘리’ 역시 각성하며 자신을 ‘할리 퀸’이라 지칭하며 서로에게 깊이 빠져든다.

무고한 시민을 죽인 죄로 재판에 오르게 된 ‘아서’
그는 최후의 심판대에서 ‘할리 퀸’과 함께 자신, ‘조커’의 이야기를 시작하는데…


In [58]:
# 코드 아이디로 감독, 배우 정보가 있는 staff 관련 태그 얻어내기
staff_id = f"{code}_staff"
staff_info = movie_info_template.find_element_by_id(staff_id)

desctiption_info = staff_info.find_element_by_css_selector('dl.desc_info')
directorsNLeadActors = desctiption_info.find_elements_by_css_selector('div')

In [59]:
# # 감독 이름, 코드 데이터 얻기
# directors = directorsNLeadActors[0].find_elements_by_css_selector('dd')

# director_set = {}
# for i in range(len(directors)):
#     directors[i].find_element_by_css_selector('a').click()
#     director_template = driver.find_elements_by_css_selector('div[tabindex="-1"]')[1]


#     director_name = director_template.find_element_by_css_selector('div.hd_layer > div').text
#     director_detail = director_template.find_element_by_css_selector('div.item_tab > div.ovf > dl.ovf')
#     director_code = director_detail.find_elements_by_css_selector('dt, dd')[1].text
#     director_set[director_code] = director_name
#     # print("director_code = " + director_code)
#     # print("director_name = " + director_set[director_code])

#     # 뒤로가기
#     director_template.find_element_by_css_selector('a.close').click()

# # 감독 딕셔너리 내용 출력
# for key, value in director_set.items():
#     print(f"{key}: {value}")

10080327: 토드 필립스 (Todd Phillips )


In [70]:
# 감독 이름, 코드 데이터 얻기2 - onClick()
directors = directorsNLeadActors[0].find_elements_by_css_selector('dd')

director_set = {}
for i in range(len(directors)):
    director_info = directors[i].find_element_by_css_selector('a')
    director_name = director_info.text
    on_click_attr = director_info.get_attribute('onClick')
    match = re.search(r"mstView\('people','(\d+)'\);", on_click_attr)
    
    if match:
        director_code = match.group(1)
        director_set[director_code] = director_name
    else:
        print("코드를 찾을 수 없습니다.")

# 감독 딕셔너리 내용 출력
for key, value in director_set.items():
    print(f"{key}: {value}")

10080327: 토드 필립스


In [74]:
lead_actors_template = directorsNLeadActors[1].find_elements_by_css_selector('dd')[0]
lead_actors_template2 = lead_actors_template.find_elements_by_css_selector('table > tbody > tr > td')[0]
lead_actors = lead_actors_template2.find_elements_by_css_selector('a')

lead_actor_set = {}
for i in range(len(lead_actors)):
    lead_actor_name = lead_actors[i].text
    on_click_attr = lead_actors[i].get_attribute('onClick')
    match = re.search(r"mstView\('people','(\d+)'\);", on_click_attr)

    if match:
        lead_actor_code = match.group(1)
        lead_actor_set[lead_actor_code] = lead_actor_name
    else:
        print("코드를 찾을 수 없습니다.")

# 감독 딕셔너리 내용 출력
for key, value in lead_actor_set.items():
    print(f"{key}: {value}")

10051376: 호아킨 피닉스
20166682: 레이디 가가


In [75]:
# 영화 상세정보 닫기
driver.find_element_by_css_selector('div.hd_layer > a.close').click()